# Phân tích dữ liệu bán hàng

Notebook này sử dụng các module `data_processor` và `utils` để phân tích dữ liệu bán hàng, tạo báo cáo và thống kê.

In [42]:
import sys
import os
import importlib

# Thêm đường dẫn module vào sys.path
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

# Import module
import data_processor
import utils

# Reload module để đảm bảo thay đổi được áp dụng
importlib.reload(data_processor)
importlib.reload(utils)

from data_processor import read_excel_file, save_to_excel, merge_dataframes
from utils import create_summary_report, save_json_report, create_directory_if_not_exists
import pandas as pd
import numpy as np
from datetime import datetime

print("✅ Đã reload module thành công")

✅ Đã reload module thành công


In [43]:
# Đọc dữ liệu bán hàng và nhân viên
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data')
sales_path = os.path.join(data_dir, 'sales.xlsx')
employees_path = os.path.join(data_dir, 'employees.xlsx')

sales_df = read_excel_file(sales_path)
employees_df = read_excel_file(employees_path)

print(f"Đã tải {len(sales_df)} bản ghi bán hàng")
print(f"Đã tải {len(employees_df)} bản ghi nhân viên")

print("\nThông tin dữ liệu bán hàng:")
print(sales_df.info())
print("\n5 dòng đầu dữ liệu bán hàng:")
sales_df.head()

✅ Đã đọc thành công file: sales.xlsx
📊 Số dòng: 50, Số cột: 9
✅ Đã đọc thành công file: employees.xlsx
📊 Số dòng: 20, Số cột: 6
Đã tải 50 bản ghi bán hàng
Đã tải 20 bản ghi nhân viên

Thông tin dữ liệu bán hàng:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Ngày bán       50 non-null     datetime64[ns]
 1   Mã sản phẩm    50 non-null     object        
 2   Tên sản phẩm   50 non-null     object        
 3   Số lượng       50 non-null     int64         
 4   Đơn giá        50 non-null     int64         
 5   Nhân viên bán  50 non-null     object        
 6   Khách hàng     50 non-null     object        
 7   Ghi chú        41 non-null     object        
 8   Thành tiền     50 non-null     int64         
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 3.6+ KB
None

5 dòng đầu dữ liệu bán hàng:


,Ngày bán,Mã sản phẩm,Tên sản phẩm,Số lượng,Đơn giá,Nhân viên bán,Khách hàng,Ghi chú,Thành tiền
0,2024-01-01,SP8805,AirPods Pro,16,47705976,Bùi Thị Hạnh,KH336,NaN,763295616
1,2024-01-05,SP8385,Sony WH-1000XM5,14,40961158,Ngô Văn Inh,KH306,Bán buôn,573456212
2,2024-01-13,SP4157,iPhone 15,10,48798540,Vũ Thị Phương,KH895,Đặt hàng online,487985400
3,2024-01-23,SP6881,Mechanical Keyboard,5,47287763,Đinh Thị Kiều,KH861,Khuyến mãi,236438815
4,2024-02-04,SP6056,Samsung Galaxy S24,5,13222094,Tôn Thị Oanh,KH742,Bán lẻ,66110470


In [44]:
# Xử lý và làm sạch dữ liệu
sales_df['Ngày bán'] = pd.to_datetime(sales_df['Ngày bán'])
sales_df['Tháng'] = sales_df['Ngày bán'].dt.strftime('%Y-%m')
sales_df['Quý'] = sales_df['Ngày bán'].dt.quarter
sales_df['Năm'] = sales_df['Ngày bán'].dt.year

# Tạo mapping từ tên nhân viên sang ID để có thể merge
employee_mapping = employees_df.set_index('Họ tên')['ID'].to_dict()
sales_df['ID_NV'] = sales_df['Nhân viên bán'].map(employee_mapping)

print(f"Đã xử lý dữ liệu: {len(sales_df)} bản ghi bán hàng")
print("\nDữ liệu sau khi xử lý:")
sales_df[['Ngày bán', 'Tên sản phẩm', 'Số lượng', 'Thành tiền', 'Tháng']].head()

Đã xử lý dữ liệu: 50 bản ghi bán hàng

Dữ liệu sau khi xử lý:


,Ngày bán,Tên sản phẩm,Số lượng,Thành tiền,Tháng
0,2024-01-01,AirPods Pro,16,763295616,2024-01
1,2024-01-05,Sony WH-1000XM5,14,573456212,2024-01
2,2024-01-13,iPhone 15,10,487985400,2024-01
3,2024-01-23,Mechanical Keyboard,5,236438815,2024-01
4,2024-02-04,Samsung Galaxy S24,5,66110470,2024-02


In [45]:
# Phân tích doanh thu
print("=== PHÂN TÍCH DOANH THU ===")

# Tổng doanh thu
total_revenue = sales_df['Thành tiền'].sum()
print(f"Tổng doanh thu: {total_revenue:,.0f} VND")

# Doanh thu theo tháng
monthly_revenue = sales_df.groupby('Tháng')['Thành tiền'].sum().sort_index()
print("\nDoanh thu theo tháng:")
for month, revenue in monthly_revenue.items():
    print(f"{month}: {revenue:,.0f} VND")

# Doanh thu theo nhân viên bán
seller_revenue = sales_df.groupby('Nhân viên bán')['Thành tiền'].agg(['sum', 'count', 'mean']).round(2)
seller_revenue = seller_revenue.sort_values('sum', ascending=False)
print("\nTop 10 nhân viên bán hàng:")
print(seller_revenue.head(10))

# Số lượng và giá trị trung bình mỗi giao dịch
avg_transaction = sales_df['Thành tiền'].mean()
print(f"\nGiá trị giao dịch trung bình: {avg_transaction:,.0f} VND")
print(f"Tổng số giao dịch: {len(sales_df)}")

=== PHÂN TÍCH DOANH THU ===
Tổng doanh thu: 14,812,220,249 VND

Doanh thu theo tháng:
2024-01: 2,061,176,043 VND
2024-02: 789,350,370 VND
2024-03: 642,179,890 VND
2024-04: 1,194,061,666 VND
2024-05: 3,089,378,157 VND
2024-06: 264,644,384 VND
2024-07: 544,239,549 VND
2024-08: 1,556,239,343 VND
2024-09: 1,105,303,856 VND
2024-10: 1,031,097,944 VND
2024-11: 766,016,330 VND
2024-12: 1,768,532,717 VND

Top 10 nhân viên bán hàng:
                       sum  count          mean
Nhân viên bán                                  
Vũ Thị Phương   1832365586      5  3.664731e+08
Trương Văn Sơn  1827973078      5  3.655946e+08
Ngô Văn Inh     1592056003      3  5.306853e+08
Bùi Thị Hạnh    1396161456      3  4.653872e+08
Lê Văn Cường    1304638290      4  3.261596e+08
Nguyễn Văn An   1301780252      5  2.603561e+08
Đỗ Văn Uy        822457497      2  4.112287e+08
Tôn Thị Oanh     819956146      4  2.049890e+08
Mai Thị Minh     626414522      2  3.132073e+08
Lý Thị Quỳnh     582180522      3  1.940602e

In [46]:
# Phân tích sản phẩm
print("=== PHÂN TÍCH SẢN PHẨM ===")

# Doanh thu theo sản phẩm
product_revenue = sales_df.groupby('Tên sản phẩm')['Thành tiền'].agg(['sum', 'count', 'mean']).round(2)
product_revenue = product_revenue.sort_values('sum', ascending=False)
print("Top 10 sản phẩm bán chạy theo doanh thu:")
print(product_revenue.head(10))

# Sản phẩm bán chạy nhất theo số lượng
product_quantity = sales_df.groupby('Tên sản phẩm')['Số lượng'].sum().sort_values(ascending=False)
print("\nTop 10 sản phẩm bán chạy theo số lượng:")
print(product_quantity.head(10))

# Phân tích giá bán
print("\nPhân tích giá bán:")
print(f"Đơn giá cao nhất: {sales_df['Đơn giá'].max():,.0f} VND")
print(f"Đơn giá thấp nhất: {sales_df['Đơn giá'].min():,.0f} VND")
print(f"Đơn giá trung bình: {sales_df['Đơn giá'].mean():,.0f} VND")

=== PHÂN TÍCH SẢN PHẨM ===
Top 10 sản phẩm bán chạy theo doanh thu:
                            sum  count          mean
Tên sản phẩm                                        
Samsung Galaxy S24   3025258907      8  3.781574e+08
Sony WH-1000XM5      2451567282      6  4.085945e+08
Gaming Mouse         2047331607      6  3.412219e+08
iPad Air             2043207763      7  2.918868e+08
MacBook Pro          1287926173      4  3.219815e+08
AirPods Pro          1086359135      5  2.172718e+08
Surface Pro          1080563437      4  2.701409e+08
Mechanical Keyboard   678803396      4  1.697008e+08
Laptop Dell XPS       587744542      4  1.469361e+08
iPhone 15             523458007      2  2.617290e+08

Top 10 sản phẩm bán chạy theo số lượng:
Tên sản phẩm
Samsung Galaxy S24     100
Gaming Mouse            87
Sony WH-1000XM5         62
iPad Air                61
MacBook Pro             44
Surface Pro             44
AirPods Pro             35
Mechanical Keyboard     24
Laptop Dell XPS         18

In [47]:
# Tạo báo cáo tổng hợp
summary_data = {
    'report_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'analysis_period': {
        'start_date': sales_df['Ngày bán'].min().strftime('%Y-%m-%d'),
        'end_date': sales_df['Ngày bán'].max().strftime('%Y-%m-%d')
    },
    'total_transactions': len(sales_df),
    'total_revenue': float(total_revenue),
    'average_transaction': float(sales_df['Thành tiền'].mean()),
    'top_product': product_revenue.index[0],
    'top_product_revenue': float(product_revenue.iloc[0]['sum']),
    'best_month': monthly_revenue.idxmax(),
    'best_month_revenue': float(monthly_revenue.max()),
    'top_sellers': seller_revenue.head(5).index.tolist(),
    'product_count': sales_df['Tên sản phẩm'].nunique(),
    'customer_count': sales_df['Khách hàng'].nunique()
}

print("Báo cáo tổng hợp:")
print(f"- Tổng giao dịch: {summary_data['total_transactions']}")
print(f"- Tổng doanh thu: {summary_data['total_revenue']:,.0f} VND")
print(f"- Giao dịch trung bình: {summary_data['average_transaction']:,.0f} VND")
print(f"- Sản phẩm bán chạy nhất: {summary_data['top_product']}")
print(f"- Tháng bán chạy nhất: {summary_data['best_month']}")
print(f"- Số sản phẩm khác nhau: {summary_data['product_count']}")
print(f"- Số khách hàng: {summary_data['customer_count']}")

Báo cáo tổng hợp:
- Tổng giao dịch: 50
- Tổng doanh thu: 14,812,220,249 VND
- Giao dịch trung bình: 296,244,405 VND
- Sản phẩm bán chạy nhất: Samsung Galaxy S24
- Tháng bán chạy nhất: 2024-05
- Số sản phẩm khác nhau: 10
- Số khách hàng: 49


In [51]:
# Lưu kết quả ra thư mục output
output_dir = os.path.join(os.path.dirname(os.getcwd()), 'output')
create_directory_if_not_exists(output_dir)

# 1. Lưu dữ liệu bán hàng đã xử lý
processed_sales_file = os.path.join(output_dir, 'processed_sales_data.xlsx')
save_to_excel(sales_df, processed_sales_file)
print(f"✅ Đã lưu dữ liệu đã xử lý: {processed_sales_file}")

# 2. Lưu báo cáo doanh thu theo nhân viên
seller_file = os.path.join(output_dir, 'sales_by_employee.xlsx')
save_to_excel(seller_revenue.reset_index(), seller_file)
print(f"✅ Đã lưu báo cáo theo nhân viên: {seller_file}")

# 3. Lưu báo cáo sản phẩm
product_file = os.path.join(output_dir, 'product_performance.xlsx')
save_to_excel(product_revenue.reset_index(), product_file)
print(f"✅ Đã lưu hiệu suất sản phẩm: {product_file}")

# 4. Lưu doanh thu theo tháng
monthly_file = os.path.join(output_dir, 'monthly_revenue.xlsx')
save_to_excel(monthly_revenue.reset_index(), monthly_file)
print(f"✅ Đã lưu doanh thu tháng: {monthly_file}")

# 5. Lưu báo cáo tổng hợp JSON
summary_file = os.path.join(output_dir, 'sales_summary.json')
save_json_report(summary_data, summary_file)
print(f"✅ Đã lưu tóm tắt JSON: {summary_file}")

# 6. Lưu báo cáo văn bản
report_file = os.path.join(output_dir, 'sales_analysis_report.txt')
create_summary_report(summary_data, report_file)
print(f"✅ Đã lưu báo cáo văn bản: {report_file}")

# 7. Lưu giao dịch có giá trị cao
high_value_transactions = sales_df[sales_df['Thành tiền'] > sales_df['Thành tiền'].quantile(0.9)]
high_value_file = os.path.join(output_dir, 'high_value_transactions.xlsx')
save_to_excel(high_value_transactions, high_value_file)
print(f"✅ Đã lưu giao dịch giá trị cao: {high_value_file}")

print(f"\n🎉 Hoàn thành! Đã tạo {7} file output từ phân tích dữ liệu bán hàng.")

📁 Thư mục đã tồn tại: d:\twan-projects\nbrunner\resources\output
💾 Đã lưu file Excel: d:\twan-projects\nbrunner\resources\output\processed_sales_data.xlsx
✅ Đã lưu dữ liệu đã xử lý: d:\twan-projects\nbrunner\resources\output\processed_sales_data.xlsx
💾 Đã lưu file Excel: d:\twan-projects\nbrunner\resources\output\sales_by_employee.xlsx
✅ Đã lưu báo cáo theo nhân viên: d:\twan-projects\nbrunner\resources\output\sales_by_employee.xlsx
💾 Đã lưu file Excel: d:\twan-projects\nbrunner\resources\output\product_performance.xlsx
✅ Đã lưu hiệu suất sản phẩm: d:\twan-projects\nbrunner\resources\output\product_performance.xlsx
💾 Đã lưu file Excel: d:\twan-projects\nbrunner\resources\output\monthly_revenue.xlsx
✅ Đã lưu doanh thu tháng: d:\twan-projects\nbrunner\resources\output\monthly_revenue.xlsx
💾 Đã lưu JSON tại: d:\twan-projects\nbrunner\resources\output\sales_summary.json
✅ Đã lưu tóm tắt JSON: d:\twan-projects\nbrunner\resources\output\sales_summary.json
📊 Đã tạo báo cáo tại: d:\twan-projec

In [50]:
# Test đơn giản cho create_summary_report
test_data = {'test': 'value', 'number': 123}
test_file = os.path.join(output_dir, 'test_report.txt')

print(f"Testing create_summary_report với file: {test_file}")
print(f"Extension: {os.path.splitext(test_file)[1]}")

try:
    result = create_summary_report(test_data, test_file)
    print(f"✅ Kết quả: {result}")
    
    # Đọc nội dung file để kiểm tra
    if os.path.exists(test_file):
        with open(test_file, 'r', encoding='utf-8') as f:
            content = f.read()
        print(f"✅ File đã được tạo. Kích thước: {len(content)} ký tự")
        print("--- Nội dung file ---")
        print(content[:200] + "..." if len(content) > 200 else content)
    else:
        print("❌ File không được tạo")
        
except Exception as e:
    print(f"❌ Lỗi: {e}")
    import traceback
    traceback.print_exc()

Testing create_summary_report với file: d:\twan-projects\nbrunner\resources\output\test_report.txt
Extension: .txt
📊 Đã tạo báo cáo tại: d:\twan-projects\nbrunner\resources\output\test_report.txt
✅ Kết quả: True
✅ File đã được tạo. Kích thước: 128 ký tự
--- Nội dung file ---
BÁO CÁO TỔNG HỢP
Thời gian tạo: 2025-06-21 21:40:27

test: value
number: 123



In [ ]:
import time
time.sleep(5)  # Đợi 1 giây để đảm bảo mọi thứ đã hoàn thành